https://docs.python.org/3/library/unittest.mock.html

[unittest.mock](https://docs.python.org/3/library/unittest.mock.html) : for stubbing dependencies and mocking side effects
-  a library for testing in Python. It allows you to replace parts of your system under test with mock objects and make assertions about how they have been used. 

In [1]:
from unittest.mock import MagicMock, Mock, patch
import pytest

In [2]:
thing = MagicMock()
thing.method = MagicMock(return_value=3)
assert thing.method(3, 4, 5, key='value') == 3

In [3]:
print(thing.method.assert_called_with(3, 4, 5, key='value'))

None


side_effect

In [5]:
mock = MagicMock(side_effect=KeyError('foo'))

In [7]:
mock

<MagicMock id='140606497955744'>

In [10]:
mock()

KeyError: 'foo'

In [11]:
try:
    mock()
except Exception as ex:
    print(str(ex))

'foo'


MagicMock()

In [13]:
values = {'a': 1, 'b': 2, 'c': 3}
def side_effect(arg):
    return values[arg]

mock = MagicMock()
mock.side_effect = side_effect
mock('a'), mock('b'), mock('c')
# (1, 2, 3)

(1, 2, 3)

In [15]:
mock.side_effect = [5, 4, 3, 2, 1]
mock(), mock(), mock()
# (5, 4, 3)

(5, 4, 3)

The `patch()` decorator / context manager makes it easy to mock classes or objects in a module under test. The object you specify will be replaced with a mock (or other object) during the test and restored when the test ends:


By default patch() will create a MagicMock for you

In [20]:
from unittest.mock import patch

import sys
sys.modules["moduleX"] = MagicMock()   # mock a module

import moduleX

@patch('moduleX.ClassName2')
@patch('moduleX.ClassName1')
def test_modx(MockClass1, MockClass2):
    moduleX.ClassName1()
    moduleX.ClassName2()
    print(str(MockClass1))
    assert MockClass1 is moduleX.ClassName1
    assert MockClass2 is moduleX.ClassName2
    print(MockClass1.called)
    assert MockClass1.called
    assert MockClass2.called

In [21]:
test_modx()

<MagicMock name='ClassName1' id='140606496995792'>
True


patch.dict()

In [27]:
foo = {'key': 'value'}
original = foo.copy()
assert foo == original
with patch.dict(foo, {'newkey': 'newvalue'}, clear=True):
    print(foo) 
    assert foo == {'newkey': 'newvalue'}

print(foo)

{'newkey': 'newvalue'}
{'key': 'value'}


In [35]:
mock = MagicMock()
mock.__str__.return_value = 'foobarbaz'
print(str(mock))
# 'foobarbaz'
print(mock.__str__.assert_called_with())

foobarbaz
None


In [36]:
mock = Mock()
mock.__str__ = Mock(return_value='wheeeeee')
str(mock)

'wheeeeee'

In [37]:
mock = MagicMock()
mock.__str__ = MagicMock(return_value='wheeeeee')
str(mock)

'wheeeeee'

auto-specing

In [40]:
from unittest.mock import create_autospec
def function(a, b, c):
    pass

mock_function = create_autospec(function, return_value='fishy')
print(mock_function(1, 2, 3))
# 'fishy'
print(mock_function.assert_called_once_with(1, 2, 3))

fishy
None


In [41]:
mock_function('wrong arguments')

TypeError: missing a required argument: 'b'

In [43]:
mock = Mock()
mock.method()
print(mock.method.assert_called())

None


configure_mock

In [44]:
mock = Mock()
attrs = {'method.return_value': 3, 'other.side_effect': KeyError}
mock.configure_mock(**attrs)
print(mock.method())
# 3
mock.other()

3


KeyError: 

In [58]:
import traceback

try:
    x = 1/10
    # raise ValueError
except ValueError:
    tb = traceback.format_exc()
except ZeroDivisionError:
    tb = traceback.format_exc()
else:
    tb = "No Exception"
finally:
    print(tb)

No Exception


In [60]:
import traceback

attrs = {'method.return_value': 3, 'other.side_effect': KeyError}
mock = Mock(some_attribute='eggs', **attrs)
print(mock.some_attribute)
# 'eggs'
print(mock.method())
# 3
try:
    mock.other()
except:
    tb = traceback.format_exc()
finally:
    print(tb)

eggs
3
Traceback (most recent call last):
  File "<ipython-input-60-baf01636020e>", line 10, in <module>
    mock.other()
  File "/usr/lib/python3.8/unittest/mock.py", line 1081, in __call__
    return self._mock_call(*args, **kwargs)
  File "/usr/lib/python3.8/unittest/mock.py", line 1085, in _mock_call
    return self._execute_mock_call(*args, **kwargs)
  File "/usr/lib/python3.8/unittest/mock.py", line 1140, in _execute_mock_call
    raise effect
KeyError



call_count

In [62]:
mock = Mock(return_value=None)
print(mock.call_count)
mock()
mock()
print(mock.call_count)

0
2


side_effect

This can either be a function to be called when the mock is called, an iterable or an exception (class or instance) to be raised.

In [64]:
mock = Mock()
mock.side_effect = Exception('Boom!')
try:
    mock()
except:
    print("Boom!")

Boom!


In [65]:
mock = Mock()
mock.side_effect = [3, 2, 1]
mock(), mock(), mock()

(3, 2, 1)

In [68]:
from unittest.mock import DEFAULT
mock = Mock(return_value=3)
def side_effect(*args, **kwargs):
    return DEFAULT

mock.side_effect = side_effect
mock()

3

In [69]:
side_effect = lambda value: value + 1
mock = Mock(side_effect=side_effect)
print(mock(3))
print(mock(-8))

4
-7
